# Python Email Exercise Ideas

Since we can't really assess any code that would involve your personal email address, here are some ideas for you to test your new skills. Please keep in mind, we can not assess these.


## Ideas

* Daily Automatic Email Reminder for your Tasks
* Webscrape some statistics from a website automatically each day and email them to yourself
* Automatically email daily/weekly/monthly reports at your work
* Have end of day messages to your friends and family be sent out at random to spread joy
* Be creative! Mix together any of the skills you've learned so far with email :)

# Podcast Newsletter App

In [ ]:
'''
IDEA:

- create a Podcast newsletter for headlines from geopolitics, finance, science, tech, and AI.
    - websites:
        - geopolitics:
            - https://www.reuters.com/world/
            - https://ground.news/
        - finance
            - https://www.reuters.com/business/
            - https://www.reuters.com/markets/ 
        - science:
            - https://ground.news/interest/science
            - https://www.sciencenews.org/
        - tech: 
            - https://www.wired.com/
            - https://ground.news/interest/tech
        - AI:
            - https://ground.news/interest/ai
            - https://www.artificialintelligence-news.com/

- newsletter that will be sent every second day at 9am (server needed?)
- set up server on Google Cloud Run

'''

In [ ]:
'''
STEPS:

1. set up web scraper -> headlines, text
2. set up LLM (perplexity) integration -> short explainers for biggest headlines (let it choose which is biggest headline)
3. finalize text and add image as part of email
4. set up emailing infrastructure
5. integrate structure into Cloud Run (Docker)

'''

In [ ]:
# TODO add error handling

import torch
import datetime
import random
import os
import requests
import bs4
import smtplib
import getpass
from PIL import Image

from huggingface_hub import hf_hub_download
from llama_cpp import Llama

In [5]:
# - geopolitics:
    # - https://www.reuters.com/world/
    # - https://ground.news/
# - finance
    # - https://www.reuters.com/business/
    # - https://www.reuters.com/markets/ 
# - science:
    # - https://ground.news/interest/science
    # - https://www.sciencenews.org/
# - tech: 
    # - https://www.wired.com/
    # - https://ground.news/interest/tech
# - AI:
    # - https://ground.news/interest/ai
    # - https://www.artificialintelligence-news.com/

In [69]:
### 1 ### scrape headlines and first few lines

# FIX needed?

# headlines_and_text = {} # dict for headlines and text
categories = ['geopolitics', 'science']
# categories = ['geopolitics', 'finance', 'science', 'tech', 'ai']

In [7]:
# request websites

# TODO use Forbes instead of reuters
# reuters_geo = requests.get('https://www.reuters.com/world/')
ground_geo = requests.get('https://ground.news/')
# reuters_bus = requests.get('https://www.reuters.com/business/')
# reuters_fin = requests.get('https://www.reuters.com/markets/')
news_sci = requests.get('https://www.sciencenews.org/')
ground_sci = requests.get('https://ground.news/interest/science')
wired_tech = requests.get('https://www.wired.com/')
ground_tech = requests.get('https://ground.news/interest/tech')
ground_ai = requests.get('https://ground.news/interest/ai')
news_ai = requests.get('https://www.artificialintelligence-news.com/')

In [38]:
# function for getting headlines:
# TODO add the images
# TODO how to get rid of extra symbols in text?

def get_headline(request, category, request_2=None):

    def get_number_headlines(headlines, category):
        mapping = {
            "geopolitics": 1,
            "finance": 1,
            "science" : 1,
            "tech": 1,
            "ai": 1
        }

        n = mapping[category]

        return set(random.sample([headline.getText().strip() for headline in headlines], n))

    if request_2 is None:
        soup = bs4.BeautifulSoup(request.text, 'lxml')
        headlines = soup.find_all('h4')

        return list(get_number_headlines(headlines, category))
    else:
        soup_1 = bs4.BeautifulSoup(request.text, 'lxml')
        headlines_1 = soup_1.find_all('h4')
        headlines_1 = get_number_headlines(headlines_1, category)
    
        soup_2 = bs4.BeautifulSoup(request_2.text, 'lxml')
        headlines_2 = soup_2.find_all('h4')
        headlines_2 = get_number_headlines(headlines_2, category)
        headlines_1.update(headlines_2)
        return list(headlines_1)

In [39]:
# getting all headlines

headlines_geo = get_headline(ground_geo, "geopolitics")
headlines_sci = get_headline(ground_sci, "science", news_sci)

# todo finish all other websites


In [40]:
mapping_lists = {
    "geopolitics": headlines_geo,
    # "finance": headlines_fin,
    "science": headlines_sci,
    # "tech": headlines_tech,
    # "ai": headlines_ai
}

In [43]:
#### MODEL ####

repo_id = "QuantFactory/Qwen3-4B-GGUF"
filename = "Qwen3-4B.Q4_K_M.gguf"  # e.g., Q8_0 for higher quality
model_path = hf_hub_download(repo_id=repo_id, filename=filename)

Qwen3-4B.Q4_K_M.gguf:   0%|          | 0.00/2.72G [00:00<?, ?B/s]

In [58]:
# Plain completion (no chat roles)
llm = Llama(
    model_path=model_path,
    n_ctx=8192,
    n_threads=os.cpu_count(),      # use all CPU cores
    n_threads_batch=os.cpu_count(),# speed up prompt eval
    n_batch=1536,                  # increase prompt batch for faster prefill (tune with VRAM/RAM)
    # n_gpu_layers=0,                # set >0 or -1 to offload to GPU if CUDA/Metal build is available
    verbose=False,
)

llama_context: n_ctx_per_seq (8192) < n_ctx_train (40960) -- the full capacity of the model will not be utilized
ggml_metal_init: skipping kernel_get_rows_bf16                     (not supported)
ggml_metal_init: skipping kernel_set_rows_bf16                     (not supported)
ggml_metal_init: skipping kernel_mul_mv_bf16_f32                   (not supported)
ggml_metal_init: skipping kernel_mul_mv_bf16_f32_c4                (not supported)
ggml_metal_init: skipping kernel_mul_mv_bf16_f32_1row              (not supported)
ggml_metal_init: skipping kernel_mul_mv_bf16_f32_l4                (not supported)
ggml_metal_init: skipping kernel_mul_mv_bf16_bf16                  (not supported)
ggml_metal_init: skipping kernel_mul_mv_id_bf16_f32                (not supported)
ggml_metal_init: skipping kernel_mul_mm_bf16_f32                   (not supported)
ggml_metal_init: skipping kernel_mul_mm_id_bf16_f16                (not supported)
ggml_metal_init: skipping kernel_flash_attn_ext_bf16_h64 

In [ ]:
# TODO text must be in HTML

def get_newsletter_text(newsletter_text, category):
    date = datetime.datetime.now().strftime("%B %d, %Y")
    topics = mapping_lists[category]

    for topic in topics:
        newsletter_text += f"{topic}\n"
        # prepare the model input
        prompt = f"""
            Write on the news happening today. Topic: {topic}. Category: {category}. Today's date: {date}. 
            Write exactly six sentences: the first five should cover the news, and the last should explain why it is relevant right now.
            Return only the sentences — no introduction, no closing, no extra text, no markdown.
        """

        messages = [
            {"role": "system", "content": "You are a journalist writing a short paragraph for a daily newsletter."},
            {"role": "user", "content": f"{prompt}"},
        ]

        # print(f'Prompt: {prompt}')

        out = llm.create_chat_completion(
            messages=messages,
            max_tokens=600, # trim for faster return
            temperature=0.25,
            top_p=0.95,
            top_k=40,
            stop=["</think>"]
        )
        print(out["choices"][0]["message"]["content"])
        response = out["choices"][0]["message"]["content"].strip()

        print(f'QWEN: {response}')

        newsletter_text += f"{response}\n\n"

    return newsletter_text

In [ ]:
# TODO text must be in HTML
newsletter_text = "Welcome to the daily newsletter from the Do You Belong? Podcast!\nWe provide you daily with the freshest and most interesting news from politics, finance, science, technology, and AI. No need to be bored or doomscrolling on your ride to work, check out the newsletter!"

for category in categories:
    print(f"Getting text for cat. - {category}")
    newsletter_text += f"\n\n{category.upper()}\n\n"
    newsletter_text = get_newsletter_text(newsletter_text, category)
    print(f"text after:\n\n{newsletter_text}")


Getting text for cat. - geopolitics
<think>
Okay, the user wants me to write a news paragraph about Trump being impatient with Russia on Ukraine, as per VP Vance. The date is September 26, 2025. They specified exactly four sentences, with the first three covering the news and the fourth explaining relevance.

First, I need to confirm the key points: Trump's impatience, VP Vance's statement, and the context of the Ukraine-Russia situation. I should start by stating the main news, then mention the source (VP Vance), then the implications. The fourth sentence should tie it to current relevance, maybe linking to ongoing tensions or the importance of U.S.-Russia relations.

Wait, the user mentioned "geopolitics" as the category. So I need to ensure the sentences are concise and fit into that category. Also, the user wants only the sentences, no extra text. Let me check if I have the correct date and that the statements are accurate. Maybe mention the specific actions Trump is taking, like p

In [ ]:
#### EMAILING ARCHITECTURE ####

import stmplib
import getpass # FIX in cloud will use native passwords

In [ ]:
stmp_object = stmplib.STMP('stmp.gmail.com', 587)

In [ ]:
# initializing server and connection
stmp_object.ehlo()

# init security (handshake) and auth layer TLS
stmp_object.starttls()

In [ ]:
email = getpass.getpass("Enter your email: ")
password = getpass.getpass("Enter your password: ")

smtp_object.login(email,password)

In [ ]:
from_address = getpass.getpass("Enter your email: ")
to_address = getpass.getpass("Enter the email of the recipient: ")

date = datetime.datetime.now().strftime('%B %d, %Y')

subject = f"Do You Belong? Newsletter - {date}"

# TODO convert text into email message
message = input("Type out the message you want to send: ")
msg = "Subject: " + subject + '\n' + message
smtp_object.sendmail(from_address,to_address,msg)

In [ ]:
smtp_object.quit()